# Randomized Benchmarking: Interleaved RB

In [ ]:
# Needs in terminal:
# $ quilc -S
# $ qvm -S

import numpy as np

from pyquil.api import get_benchmarker, get_qc
from forest.benchmarking.randomized_benchmarking import *
from forest.benchmarking.plotting import plot_figure_for_fit

from pyquil.gates import *
from pyquil import Program


%matplotlib inline

# You need a Quantum Computer and a Benchmarker

In [ ]:
bm = get_benchmarker()
qc = get_qc("9q-square-noisy-qvm", noisy=True)

# Setup Both RB experiments

In [ ]:
# Choose your parameters
qubit_groups = [(0, 1)]
depths = 3 * 2 ** np.arange(4, dtype=np.uint8)
num_sequences = 25
depths = [d for d in depths for _ in range(num_sequences)]

# Intialize and partially populate a DataFrame to summarize the experiment.
rb_expts = generate_rb_experiments(bm, qubit_groups, depths)
inter_expts = generate_rb_experiments(bm, qubit_groups, depths, interleaved_gate=Program(CNOT(0,1)))

# Run Standard RB experiment

In [ ]:
# Run the RB Sequences on a QuantumComputer
num_shots=100
rb_results = acquire_rb_data(qc, rb_expts, num_shots)

# Calculate a fit to a decay curve
stats = get_stats_by_qubit_group(qubit_groups, rb_results)[qubit_groups[0]]
fit = fit_rb_results(depths, stats['expectation'], stats['std_err'], num_shots)

# Extract rb decay parameter
rb_decay = fit.params['decay'].value
rb_decay_error = fit.params['decay'].stderr

# Plot
fig, axs = plot_figure_for_fit(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")

# Run Interleaved RB experiment

In [ ]:
# Run the RB Sequences on a QuantumComputer
num_shots=100
inter_results = acquire_rb_data(qc, inter_expts, num_shots)

# Calculate a fit to a decay curve
stats = get_stats_by_qubit_group(qubit_groups, inter_results)[qubit_groups[0]]
fit = fit_rb_results(depths, stats['expectation'], stats['std_err'], num_shots)

# Extract irb decay parameter
irb_decay = fit.params['decay'].value
irb_decay_error = fit.params['decay'].stderr

# Plot
fig, axs = plot_figure_for_fit(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")

In [ ]:
print(rb_decay)
print(irb_decay)

# Average Clifford gate fidelity, and interleaved gate fidelity

In [ ]:
print(rb_decay_to_gate_fidelity(rb_decay, 4))
gate_fidelity = 1 - irb_decay_to_gate_infidelity(irb_decay, rb_decay, 4)
print(gate_fidelity)

# Interleaved gate fidelity bounds

In [ ]:
bounds = interleaved_gate_fidelity_bounds(irb_decay, rb_decay, 4)
print(bounds)

In [ ]:
assert(bounds[0] < gate_fidelity and gate_fidelity < bounds[1])

# Improve gate fidelity bounds with unitarity experiment. EXTREMELY SLOW

In [ ]:
num_shots = 25

expts = generate_unitarity_experiments(bm, qubit_groups, depths, num_sequences)

results = acquire_rb_data(qc, expts, num_shots)
stats = get_stats_by_qubit_group(qubit_groups, results)[qubit_groups[0]]
fit = fit_unitarity_results(depths, stats['expectation'], stats['std_err'])

# plot the raw data, point estimate error bars, and fit
fig, axs = plot_figure_for_fit(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['decay'].value

## This may result in NaN depending on the outcome of the unitarity and difference between rb and irb decays. Getting better estimates of each helps prevent this

In [ ]:
better_bounds = interleaved_gate_fidelity_bounds(irb_decay, rb_decay, 4, unitarity)
print(better_bounds)